In [1]:
# Lemma. max_cluster for NC CD is either 11 or 12. 

# Theorem. max_cluster for NC CD is 11. 
# Proof idea. We show that 12 clusters is not achievable.

state = 'NC'
district_type = 'CD'

filepath = 'C:\\districting-data-2020\\'
filename = state + '_county.json'

from gerrychain import Graph
GC = Graph.from_json( filepath + filename )

In [2]:
# Facts:
#  1. We know that Wake and Mecklenburg County must belong to clusters of size >=2,
#     because each of their populations is more than U. 

from util import update_attributes, get_k_L_U
update_attributes(GC, state)   
(k,L,U) = get_k_L_U(GC, state, district_type)

for i in GC.nodes:
    name = GC.nodes[i]['NAME20']
    if name == 'Guilford':
        g = i
    elif name == 'Mecklenburg':
        m = i
    elif name == 'Forsyth':
        f = i
    elif name == 'Wake':
        w = i
        
print("Mecklenberg County has population:",GC.nodes[m]['TOTPOP'])
print("Wake County has population:",GC.nodes[w]['TOTPOP'])

Starting NC with k = 14 and deviation = 0.01
Thus, we have L = 741943 and U = 749398
Mecklenberg County has population: 1115482
Wake County has population: 1129410


In [3]:
#  2. Further, the Wake and Mecklenburg clusters must be distinct, since otherwise 
#     they will belong to same cluster of size >= 4, since the population-weighted 
#     distance between them is more than 3*U, which leaves only k-4 = 10 size units.
#     In this case, the max number of clusters is at most 1 + 10 = 11.

import networkx as nx
DG = nx.DiGraph(GC)

for i,j in DG.edges:
    DG.edges[i,j]['weight'] = GC.nodes[i]['TOTPOP']
    
dist = nx.shortest_path_length(DG, weight='weight', source=m)
distance = dist[w] + GC.nodes[w]['TOTPOP']

print("( population-weighted distance from Mecklenburg to Wake ) / U =",distance/U)

( population-weighted distance from Mecklenburg to Wake ) / U = 3.6351471447748724


In [4]:
# 3. So, if we are to have 12 clusters, then Wake and Mecklenburg must be in distinct size-2 clusters, 
#    and all remaining clusters must have size one. 

In [5]:
# 4. Meanwhile, Guilford County must be in a different cluster than Wake and Mecklenburg,
#    because the population-weighted distance from Guilford to them is more than 2*U.

dist = nx.shortest_path_length(DG, weight='weight', source=g)
distance = dist[w] + GC.nodes[w]['TOTPOP']
print("( population-weighted distance from Guilford to Wake ) / U =",distance/U)

distance = dist[m] + GC.nodes[m]['TOTPOP']
print("( population-weighted distance from Guilford to Mecklenburg ) / U =",distance/U)

( population-weighted distance from Guilford to Wake ) / U = 2.514858593164113
( population-weighted distance from Guilford to Mecklenburg ) / U = 2.742355864307084


In [6]:
# 5. Also, Forsyth County must be in yet another distinct cluster, by same arguments.

dist = nx.shortest_path_length(DG, weight='weight', source=f)
distance = dist[w] + GC.nodes[w]['TOTPOP']
print("( population-weighted distance from Forsyth to Wake ) / U =",distance/U)

distance = dist[m] + GC.nodes[m]['TOTPOP']
print("( population-weighted distance from Forsyth to Mecklenburg ) / U =",distance/U)

distance = dist[g] + GC.nodes[g]['TOTPOP']
print("( population-weighted distance from Forsyth to Guilford ) / U =",distance/U)

( population-weighted distance from Forsyth to Wake ) / U = 2.36248428738801
( population-weighted distance from Forsyth to Mecklenburg ) / U = 2.29781638061484
( population-weighted distance from Forsyth to Guilford ) / U = 1.2328415608261565


In [7]:
# 6. Finally, we enumerate all possible (disjoint) clusters for
#       - Guilford of size 1
#       - Mecklenburg of size 2
#       - Forsyth of size 1
#    And show that no combination of them is compatible with getting 9 other clusters from the rest of the state.

In [8]:
import gurobipy as gp
from gurobipy import GRB
from separation import find_fischetti_separator

In [9]:
def separator_callback(m, where):
    
    if where == GRB.Callback.MIPSOL:
        xval = m.cbGetSolution(m._x)
        DG = m._DG
        b = m._root
        
        # vertices in the cluster
        S = [ v for v in DG.nodes if xval[v] > 0.5 ]
            
        for component in nx.strongly_connected_components( DG.subgraph(S) ):

            if b in component: 
                continue

            # find some vertex "a" that has largest population in this component
            maxp = max( DG.nodes[v]['TOTPOP'] for v in component )
            a = [ v for v in component if DG.nodes[v]['TOTPOP'] == maxp ][0]

            # get minimal a,b-separator and add lazy cut
            C = find_fischetti_separator(DG, component, b)
            m.cbLazy( m._x[a] <= gp.quicksum( m._x[c] for c in C) )

In [10]:
def enumerate_clusters_of_given_size(DG, L, U, root, size):
    
    m = gp.Model()
    m.Params.OutputFlag = 0
    
    m._x = m.addVars(DG.nodes, vtype=GRB.BINARY)
    m._x[root].LB = 1
    
    # population balance
    m.addConstr( gp.quicksum( DG.nodes[i]['TOTPOP'] * m._x[i] for i in DG.nodes ) >= size * L )
    m.addConstr( gp.quicksum( DG.nodes[i]['TOTPOP'] * m._x[i] for i in DG.nodes ) <= size * U )
    
    # No objective
    m.setObjective( 0.0, GRB.MINIMIZE )
    
    # Find the N best solutions
    N = 1000000
    m.Params.PoolSearchMode = 2 # finds N best solutions
    m.Params.PoolSolutions = N

    # Solve
    m.Params.IntFeasTol = 1e-7
    m.Params.FeasibilityTol = 1e-7
    m.Params.LazyConstraints = 1
    m._DG = DG
    m._root = root
    m._callback = separator_callback
    m.optimize(m._callback)
    
    # retrieve clusters
    clusters = list()

    for sol in range(m.SolCount):
        m.Params.SolutionNumber = sol
        cluster = [ i for i in DG.nodes if m._x[i].Xn > 0.5 ]
        clusters.append(cluster)

    #print("Number of clusters =",len(clusters)) 
    return clusters

In [11]:
from cluster import is_within_window 

# Do all components of GC have a population that is within 
#   an integer multiple of the population balance window?
def ok(GC, L, U, k, cluster):
    set_cluster = set(cluster)
    complement = [ i for i in GC.nodes if i not in set_cluster ]
    for component in nx.connected_components( GC.subgraph(complement) ):
        population = sum( GC.nodes[i]['TOTPOP'] for i in component )
        if not is_within_window(population, L, U):
            return False
    return True

# Only keep clusters for which GC-cluster has components with 'good' population
def filter_clusters(GC, L, U, k, clusters):
    filtered_clusters = list()
    for cluster in clusters:
        if ok(GC, L, U, k, cluster):
            filtered_clusters.append(cluster)
    return filtered_clusters

In [12]:
clusterings = list()
            
# Guilford clusters
gclusters = enumerate_clusters_of_given_size(DG, L, U, root=g, size=1)

for gcluster in gclusters:

    # Mecklenburg clusters
    Vm = [ i for i in GC.nodes if i not in gcluster ]
    mclusters = enumerate_clusters_of_given_size(DG.subgraph(Vm), L, U, root=m, size=2)
    mclusters = filter_clusters(GC.subgraph(Vm), L, U, k-1, mclusters)                                         

    for mcluster in mclusters:

        # Forsyth clusters
        Vf = [ i for i in GC.nodes if i not in gcluster + mcluster ]
        fclusters = enumerate_clusters_of_given_size(DG.subgraph(Vf), L, U, root=f, size=1)
        fclusters = filter_clusters(GC.subgraph(Vf), L, U, k-3, fclusters)      

        for fcluster in fclusters:
            clusterings.append( [gcluster, mcluster, fcluster] )

print("Guilford cluster, Mecklenburg cluster, Forsyth cluster")
for p in range(len(clusterings)):
    clustering = clusterings[p]
    print("Possible clustering #",p,"is",clustering)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-27
Guilford cluster, Mecklenburg cluster, Forsyth cluster
Possible clustering # 0 is [[5, 44, 53, 76], [23, 32, 62, 74], [4, 29, 33, 34, 48, 66, 78, 80, 81, 84, 86, 89]]
Possible clustering # 1 is [[5, 44, 53, 76], [23, 32, 62, 74], [29, 33, 34, 48, 63, 78, 80, 81, 84, 86, 89]]
Possible clustering # 2 is [[5, 44, 53, 76], [15, 32, 35, 38, 62], [23, 34, 37, 70, 78, 80, 89]]
Possible clustering # 3 is [[5, 44, 53, 76], [15, 32, 35, 38, 62], [34, 48, 59, 78, 80, 84, 89]]
Possible clustering # 4 is [[5, 44, 53, 76], [15, 32, 35, 38, 62], [23, 34, 50, 59, 80, 89]]
Possible clustering # 5 is [[5, 44, 53, 76], [15, 32, 35, 62, 81], [23, 34, 50, 59, 72, 89, 99]]
Possible clustering # 6 is [[5, 44, 53, 76], [32, 70, 74], [4, 29, 33, 34, 48, 66, 78, 80, 81, 84, 86, 89]]
Possible clustering # 7 is [[5, 44, 53, 76], [32, 70, 74], [34, 35, 38, 63, 78, 80, 81, 89]]
Possible clustering # 8 is [[5, 44, 53, 76], [32

In [13]:
from cluster import max_cluster_main

# for each possible clustering combination for Guilford/Mecklenburg/Forsyth do:
for p in range(len(clusterings)):
    
    clustering = clusterings[p]
    print("Trying possible clustering #",p,"which is",clustering)
    
    [gcluster, mcluster, fcluster] = clustering
    used = gcluster + mcluster + fcluster
    VC = [ i for i in GC.nodes if i not in used ]
    
    # possible to get 9 clusters from the rest of the state?
    
    # first, try a 10-min time limit
    results = max_cluster_main(GC.subgraph(VC), L, U, k=10, max_t=2, restarts=1, time_limit=600)
    
    if results['cluster_UB'] >= 8.5:
        
        # next, try a 1-day time limit
        results = max_cluster_main(GC.subgraph(VC), L, U, k=10, max_t=2, restarts=1, time_limit=24*3600)
        
        if results['cluster_UB'] >= 8.5:
            
            print("It seems possible to get 9 other clusters!!!")
            print("We have clusters:",results['clusters'])
            print("With sizes:",results['sizes'])
            break


Trying possible clustering # 0 which is [[5, 44, 53, 76], [23, 32, 62, 74], [4, 29, 33, 34, 48, 66, 78, 80, 81, 84, 86, 89]]
Initially, cluster_UB = 9

****************************
Heuristic iteration # 0
****************************
carved cluster sizes = 1, 1, 1, 1, 3, 3, 
carved LB = 6
carved cut edges = 52
cut edges -= 4
clusters += 1 ( w/ cut edges += 28 )
cut edges -= 15
t = 2 -> #clusters, #cut edges = 7 61
new incumbent!

********************************************************
After local search, # clusters, #cut edges = 7 61
********************************************************

Set parameter TimeLimit to value 600
Set parameter IntFeasTol to value 1e-07
Set parameter FeasibilityTol to value 1e-07
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 22288 rows, 34800 columns and 142377 nonzeros
Model fingerprint: 0xd01a45b9
Variable types: 28320 continuous, 6480 integer (640

   587   382 infeasible   12         6.00000    9.00000  50.0%   567   40s
  1018   564    7.00000   41   65    6.00000    9.00000  50.0%   519   51s
  1078   567    7.00000   48  121    6.00000    9.00000  50.0%   527   55s
  1082   570    9.00000   23  206    6.00000    9.00000  50.0%   525   61s
  1088   574    9.00000   34  127    6.00000    9.00000  50.0%   522   65s
  1091   581    9.00000   11  120    6.00000    9.00000  50.0%   566   71s
  1107   589    9.00000   13  113    6.00000    9.00000  50.0%   572   75s
  1130   601    9.00000   16   96    6.00000    9.00000  50.0%   585   80s
  1165   613    8.00000   18  113    6.00000    9.00000  50.0%   583   85s
  1240   597    8.93231   16  126    6.00000    9.00000  50.0%   587   90s
  1290   597    8.59746   21  122    6.00000    9.00000  50.0%   596   95s
  1328   628    7.00000   29   51    6.00000    9.00000  50.0%   600  100s
  1414   601    8.80148   22  187    6.00000    9.00000  50.0%   597  107s
  1488   633    8.71007  

  5417   743    8.96348   42   86    7.00000    9.00000  28.6%   441  168s
  5665   695 infeasible   36         7.00000    9.00000  28.6%   448  174s
  5922   665    8.19847   25  155    7.00000    9.00000  28.6%   455  180s
  6190   686 infeasible   20         7.00000    9.00000  28.6%   462  187s
  6536   700    9.00000   31   62    7.00000    9.00000  28.6%   465  196s
  6854   716 infeasible   38         7.00000    9.00000  28.6%   467  222s
  7005   750 infeasible   37         7.00000    9.00000  28.6%   469  234s
  7232   802 infeasible   33         7.00000    9.00000  28.6%   476  245s
  7569   840    8.99205   30   94    7.00000    9.00000  28.6%   479  256s
  7798   929    9.00000   21  134    7.00000    9.00000  28.6%   483  265s
  8281  1011 infeasible   37         7.00000    9.00000  28.6%   483  277s
  8701  1072 infeasible   33         7.00000    9.00000  28.6%   488  289s
  9064  1083    8.00000   28  105    7.00000    9.00000  28.6%   492  303s
  9526  1086    8.25881  

  6652  1190    9.00000   27  101    6.00000    9.00000  50.0%   481  303s
  6749  1304    8.63582   35   88    6.00000    9.00000  50.0%   482  315s
  7158  1412 infeasible   37         6.00000    9.00000  50.0%   489  327s
  7585  1497    8.61561   24  109    6.00000    9.00000  50.0%   502  340s
  7984  1564 infeasible   28         6.00000    9.00000  50.0%   517  355s
  8189  1695 infeasible   34         6.00000    9.00000  50.0%   520  368s
  8567  1778 infeasible   27         6.00000    9.00000  50.0%   534  380s
  8936  1891    7.00000   40   84    6.00000    9.00000  50.0%   539  391s
  9331  1984    8.00000   23  147    6.00000    9.00000  50.0%   548  403s
  9703  2061    8.61999   28   60    6.00000    9.00000  50.0%   557  415s
  9991  2149 infeasible   33         6.00000    9.00000  50.0%   560  426s
 10305  2248    8.00000   24  112    6.00000    9.00000  50.0%   572  441s
 10780  2302    8.70161   31  176    6.00000    9.00000  50.0%   584  452s
 10991  2403    8.60481  

  6702   397    7.00000   23  102    6.00000    7.00000  16.7%   555  207s
  7104   252    7.00000   20  115    6.00000    7.00000  16.7%   548  215s
  7523   129    7.00000   23  112    6.00000    7.00000  16.7%   542  223s
  8038    84    7.00000   37  121    6.00000    7.00000  16.7%   538  231s
  8395    33    7.00000   31   99    6.00000    7.00000  16.7%   537  239s
  8624     0 infeasible   31         6.00000    7.00000  16.7%   537  240s

Cutting planes:
  Gomory: 7
  Lift-and-project: 2
  Cover: 1408
  Implied bound: 217
  Clique: 5
  MIR: 180
  StrongCG: 9
  Flow cover: 458
  GUB cover: 1
  Inf proof: 44
  Zero half: 1
  Network: 400
  RLT: 12
  Relax-and-lift: 13

Explored 8679 nodes (4723638 simplex iterations) in 241.03 seconds (324.15 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 6 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.000000000000e+00, best bound 6.000000000000e+00, gap 0.0000%
*************************************

...

   740   454    8.00000   22  104    6.00000    9.00000  50.0%   598   50s
   975   547 infeasible   24         6.00000    9.00000  50.0%   605   60s
  1043   551    9.00000   11  151    6.00000    9.00000  50.0%   602   67s
  1045   553    8.92658   20  198    6.00000    9.00000  50.0%   601   71s
  1049   555    8.00000   16  128    6.00000    9.00000  50.0%   599   76s
  1054   562    9.00000   14  144    6.00000    9.00000  50.0%   668   83s
  1056   563    9.00000   15  241    6.00000    9.00000  50.0%   671   88s
  1060   562    9.00000   16  188    6.00000    9.00000  50.0%   675   90s
  1076   571    7.83650   18  102    6.00000    9.00000  50.0%   678   95s
  1120   592    9.00000   23   97    6.00000    9.00000  50.0%   671  100s
  1220   551 infeasible   29         6.00000    9.00000  50.0%   675  107s
H 1278   519                       7.0000000    9.00000  28.6%   686  111s
  1341   497 infeasible   38         7.00000    9.00000  28.6%   686  115s
  1444   497 infeasible  

  4136   311    8.00000   33  228    8.00000    9.00000  12.5%   726  256s
  4140   314    8.00000   31  191    8.00000    9.00000  12.5%   725  261s
  4144   316    8.00000   36  163    8.00000    9.00000  12.5%   725  266s
  4148   319    8.14776   30  152    8.00000    9.00000  12.5%   724  270s
  4151   326    9.00000   26  166    8.00000    9.00000  12.5%   384  277s
  4159   325    9.00000   27  146    8.00000    9.00000  12.5%   386  280s
  4176   329    9.00000   29  132    8.00000    9.00000  12.5%   388  285s
  4200   337    9.00000   32  134    8.00000    9.00000  12.5%   392  290s
  4234   330 infeasible   34         8.00000    9.00000  12.5%   397  295s
  4299   327    9.00000   40  120    8.00000    9.00000  12.5%   404  303s
  4416   277    9.00000   46   84    8.00000    9.00000  12.5%   412  309s
  4523   253    9.00000   32  157    8.00000    9.00000  12.5%   422  312s
  4605   228    9.00000   45   72    8.00000    9.00000  12.5%   430  317s
  4729   209    9.00000  

 15064  3922 infeasible   49         7.00000    9.00000  28.6%   474  430s
 15542  4095    8.00000   48   58    7.00000    9.00000  28.6%   473  441s
 16229  4313    8.93402   36  103    7.00000    9.00000  28.6%   469  451s
 17079  4441    9.00000   30   98    7.00000    9.00000  28.6%   468  462s
 17567  4550 infeasible   36         7.00000    9.00000  28.6%   470  473s
 17836  4786    9.00000   30  132    7.00000    9.00000  28.6%   471  484s
 18723  5092    8.65230   36   84    7.00000    9.00000  28.6%   467  494s
 19621  5329    8.00000   33   92    7.00000    9.00000  28.6%   463  505s
 20403  5391 infeasible   33         7.00000    9.00000  28.6%   463  514s
 20699  5392    8.00000   43  128    7.00000    9.00000  28.6%   464  520s
 20708  5398    8.00000   52  109    7.00000    9.00000  28.6%   464  525s
 20712  5401    8.00000   47  175    7.00000    9.00000  28.6%   464  530s
 20717  5409    9.00000   29  133    7.00000    9.00000  28.6%   467  535s
 20737  5416    9.00000  

 16392  1088    9.00000   42  117    8.00000    9.00000  12.5%   632  569s
 17217  1073    9.00000   43  104    8.00000    9.00000  12.5%   631  585s
 17677  1120 infeasible   44         8.00000    9.00000  12.5%   635  602s
 18434  1127    9.00000   51  112    8.00000    9.00000  12.5%   638  618s
 19337  1074    9.00000   38  110    8.00000    9.00000  12.5%   636  633s
 20003  1138    9.00000   29  163    8.00000    9.00000  12.5%   639  646s
 20331  1137    9.00000   34   64    8.00000    9.00000  12.5%   646  661s
 20873  1138    9.00000   41  132    8.00000    9.00000  12.5%   652  670s
 20878  1141    9.00000   47  118    8.00000    9.00000  12.5%   652  675s
 20884  1145    9.00000   22  157    8.00000    9.00000  12.5%   652  680s
 20889  1152    9.00000   27  162    8.00000    9.00000  12.5%   655  687s
 20903  1159    9.00000   30  105    8.00000    9.00000  12.5%   654  690s
 20933  1194    9.00000   33  100    8.00000    9.00000  12.5%   654  695s
 21042  1267    9.00000  

  2201   840 infeasible   18         7.00000    9.00000  28.6%  70.0  240s
  2227   841    9.00000   20  126    7.00000    9.00000  28.6%  83.2  245s
  2264   834    9.00000   23  134    7.00000    9.00000  28.6%  97.2  251s
  2289   824    9.00000   24  125    7.00000    9.00000  28.6%   109  258s
  2294   829    8.76810   25  115    7.00000    9.00000  28.6%   111  260s
  2324   829     cutoff   28         7.00000    9.00000  28.6%   120  269s
  2331   830    8.00000   30   65    7.00000    9.00000  28.6%   121  271s
  2399   815    8.00000   21   96    7.00000    9.00000  28.6%   141  276s
  2487   786    9.00000   20  133    7.00000    9.00000  28.6%   168  280s
  2591   769    8.72639   24  123    7.00000    9.00000  28.6%   197  286s
  2745   651 infeasible   30         7.00000    8.00000  14.3%   229  292s

Cutting planes:
  Lift-and-project: 8
  Cover: 7
  Clique: 1
  MIR: 43
  Flow cover: 86
  Inf proof: 2
  Zero half: 1
  Network: 57
  Relax-and-lift: 4

Explored 2931 nodes (